In [60]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
 
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
 
use_cuda = torch.cuda.is_available()
 
MAX_LENGTH = 100

In [61]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [77]:
#make dict
 
SOS_token = 0
EOS_token = 1
UNKNOWN_token = 2
class Lang :
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"UNKNOWN"}
        self.word2count = {}
        self.n_words = 3 #count SOS and EOS and UNKWON
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [78]:
#Turn a Unicode stirng to plain ASCII
 
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
 
# Lowercase, trim, and remove non-letter characters
 
def normalizeString(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣 ^☆; ^a-zA-Z.!?]+')
    result = hangul.sub('', s)
#     s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"([.!?])", r" \1", s)
#     s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    
    return result

In [79]:
def readText():
    print("Reading lines...")
    
    inputs = open('source_test.txt', encoding='utf-8').read().strip().split('\n')
    outputs = open('target_test.txt', encoding='utf-8').read().strip().split('\n')
 
    inputs = [normalizeString(s) for s in inputs]
    outputs = [normalizeString(s) for s in outputs]
    
    inp = Lang('input')
    outp = Lang('output')
    
    pair = []
    for i in range(len(inputs)):
        pair.append([inputs[i], outputs[i]])
    return inp, outp, pair

In [80]:
def prepareData():
    input_lang, output_lang, pairs = readText()
    print("Read %s sentence pairs" % len(pairs))
 
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs
 
input_lang, output_lang, pairs = prepareData()
print(random.choice(pairs))

Reading lines...
Read 209 sentence pairs
Counting words...
Counted words:
input 889
output 841
['아 잠시만 미세먼지만큼 수정 더 하긴했음 내가 수정나거로 낼까 그럼', '아 응응 그러자 아 그러면 내가 블랙보드에 올릴까 그전에도 올렸어서']


In [89]:
nn.Embedding(input_lang.n_words, hidden_size)

Embedding(889, 256)

In [21]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        result = Variable(torch.zeros(1,1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [22]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1,1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [103]:
teacher_forcing_ratio = 0.5
 
def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
 
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
 
    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]
 
    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
 
    loss = 0
 
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        # print(encoder_outputs.size(), encoder_output[0][0].size())
        encoder_outputs[ei] = encoder_output[0][0]
 
    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
 
    decoder_hidden = encoder_hidden
 
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
 
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing
 
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]
 
            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
 
            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break
 
    loss.backward()
 
    encoder_optimizer.step()
    decoder_optimizer.step()
 
    return loss.item() / target_length

In [104]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [117]:
def indexesFromSentence(lang, sentence):
     return [lang.word2index[word] for word in sentence.split(' ')]

def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    # print(indexes)
    result = Variable(torch.LongTensor(indexes).view(-1,1))
    # print(result)
    if use_cuda:
        return result.cuda()
    else:
        return result

def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    target_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

In [105]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [108]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [37]:
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

In [38]:
# load encoder and decoder model
encoder_PATH = "./encoder.pth"
decoder_PATH = "./decoder.pth"

encoder.load_state_dict(torch.load(encoder_PATH))
attn_decoder.load_state_dict(torch.load(decoder_PATH))

<All keys matched successfully>

In [43]:
def indexesFromSentence(lang, sentence):
     return [lang.word2index[word] for word in sentence.split(' ')]

def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    print(indexes)
    result = Variable(torch.LongTensor(indexes).view(-1,1))
    if use_cuda:
        return result.cuda()
    else:
        return result

def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    target_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)

In [41]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    input_variable = variableFromSentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()
    
    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]
        
    decoder_input = Variable(torch.LongTensor([[SOS_token]])) #SOS
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    
    decoder_hidden = encoder_hidden
    
    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)
    
    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder( decoder_input, decoder_hidden, encoder_outputs)
        decoder_attentions[di] = decoder_attention.data
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni.item()])
        
        decoder_input = Variable(torch.LongTensor([[ni.item()]]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input
        
    return decoded_words, decoder_attentions[:di +1]


In [51]:
def evaluateAndShowAttention(input_sequence, encoder, decoder):
    output_words, _ = evaluate(encoder, decoder , input_sequence)
    output_sentence = ' '.join(output_words)
    print(output_words)
#    print('input =', input_sequence)
#    print('output =', ' '.join(output_words))

In [58]:
evaluateAndShowAttention('따뜻한거', encoder1, attn_decoder1)

[350, 1]
100
['잇어', '중에', '안잡히지', '못하면', '못하면', '못하면', '언제', '나왓어', '지울께', '나왓어', '위에', '위에', '위에', '중에', '나왓어', '위에', '위에', '위에', '위에', '끝나긴해', '끝나긴해', '지워버릴까', '끝나긴해', '지워버릴까', '있는', '등록금도', '위에', '위에', '위에', '위에', '끝나긴해', '끝나긴해', '끝낫으', '있는', '나왓어', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '있는', '나왓어', '잇어', '위에', '위에', '위에', '위에', '끝나긴해', '끝나긴해', '지워버릴까', '끝나긴해', '지워버릴까', '끝나긴해', '지워버릴까', '있는', '잇어', '만나서', '등록금도', '있는', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '좋은듯', '있는', '잇어', '지워버릴까', '있는', '등록금도', '그래굳인데', '그러자글면', '위에', '위에', '위에', '끝나긴해', '위에', '끝나긴해', '위에', '위에', '끝나긴해', '끝나긴해', '지워버릴까', '정해야될꺼', '끝나긴해', '지워버릴까', '있는', '등록금도', '그래굳인데', '있는', '그러자글면', '위에', '나왓어']
